In [47]:
import random
import copy

In [48]:
class Individual:
    def __init__(self, genetic_code, graph, nodes=None, fitness=None):
        self.genetic_code = genetic_code
        self.graph = graph
        if nodes is None:
            self.nodes = sorted(graph.keys())
        else:
            self.nodes = nodes
        self.num_nodes = len(self.nodes)
        self.num_dominated = None
        self.total_selected = None

        if fitness is None:
            self.fitness = self.calculate_fitness()
        else:
            self.fitness = fitness

    def __str__(self):
        return f"Gen: {''.join(map(str, self.genetic_code))} | Fitness: {self.fitness} | Selected: {sum(self.genetic_code)}"

    def __repr__(self):
        return self.__str__()

    def calculate_fitness(self):
        dominated = set()
        for idx, gene in enumerate(self.genetic_code):
            if gene == 1:
                node = self.nodes[idx]
                dominated.add(node)
                for neighbor in self.graph[node]:
                    dominated.add(neighbor)
        num_dominated = len(dominated)
        total_selected = sum(self.genetic_code)

        if num_dominated == self.num_nodes:
            fitness = self.num_nodes + (self.num_nodes - total_selected)
        else:
            fitness = num_dominated

        self.total_selected = total_selected
        self.num_dominated = num_dominated

        return fitness

In [49]:
def initial_population(graph, population_size):
    population = []
    nodes = sorted(graph.keys())
    n = len(nodes)
    for _ in range(population_size):
        genetic_code = [random.randint(0, 1) for _ in range(n)]
        individual = Individual(genetic_code, graph, nodes)
        population.append(individual)
    return population

In [50]:
def roulette_selection(population):
    total_fitness = sum(individual.fitness for individual in population)
    if total_fitness == 0:
        return random.choice(population)
    winner = random.choices(population, weights=[ind.fitness for ind in population], k=1)[0]
    return winner

In [51]:
def tournament_selection(population, tournament_size):
    selected = random.sample(population, tournament_size)
    winner = max(selected, key=lambda x: (x.fitness, -x.total_selected))
    return winner

In [52]:
def crossover(parent1, parent2):
    n = len(parent1.genetic_code)
    if n < 2:
        return parent1, parent2

    break_point = random.randint(1, n - 1)
    child1_code = parent1.genetic_code[:break_point] + parent2.genetic_code[break_point:]
    child2_code = parent2.genetic_code[:break_point] + parent1.genetic_code[break_point:]

    child1 = Individual(child1_code, parent1.graph, parent1.nodes)
    child2 = Individual(child2_code, parent2.graph, parent2.nodes)

    return child1, child2

In [53]:
def mutate(individual, mutation_rate):
    if random.random() < mutation_rate:
        index = random.randrange(len(individual.genetic_code))
        individual.genetic_code[index] = 1 - individual.genetic_code[index]
        individual.fitness = individual.calculate_fitness()
    return individual

In [54]:
def ga(graph, population_size, mutation_rate, tournament_size, max_iterations, selection_method, elitism_size):
    if selection_method not in ['roulette_selection', 'tournament_selection']:
        raise ValueError("Invalid selection method")

    population = initial_population(graph, population_size)

    for generation in range(1, max_iterations + 1):
        population.sort(key=lambda x: (-x.fitness, x.total_selected))
        best_individual = population[0]
        print(f"Generation {generation}: {best_individual}")

        new_population = []
        elites = [copy.deepcopy(individual) for individual in population[:elitism_size]]
        new_population.extend(elites)

        while len(new_population) < population_size:
            if selection_method == 'roulette_selection':
                parent1 = roulette_selection(population)
                parent2 = roulette_selection(population)
            else:
                parent1 = tournament_selection(population, tournament_size)
                parent2 = tournament_selection(population, tournament_size)

            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            children = [child1, child2]
            children.sort(key=lambda x: (-x.fitness, x.total_selected))

            for child in children:
                if len(new_population) < population_size:
                    new_population.append(child)
                else:
                    break
        population = new_population

    dominating_individuals = [individual for individual in population if individual.num_dominated == individual.num_nodes]

    if dominating_individuals:
        best = max(dominating_individuals, key=lambda x: (x.fitness, -x.total_selected))
    else:
        best = max(population, key=lambda x: (x.fitness, -x.total_selected))

    print(f"Best individual after {max_iterations} iterations: {best}")

    return best.genetic_code, best.fitness

In [55]:
if __name__ == '__main__':
    g = {
        1: [2, 5],
        2: [1, 3, 5],
        3: [2, 4, 6],
        4: [3],
        5: [1, 2, 6],
        6: [3, 5]
    }

    best_code, best_fitness = ga(
        graph=g,
        population_size=50,
        mutation_rate=0.1,
        tournament_size=3,
        max_iterations=100,
        selection_method='tournament_selection',
        elitism_size=2
    )

    nodes = sorted(g.keys())
    print("Best genetic code (dominating set):", best_code)

    selected_vertices = [nodes[i] for i, bit in enumerate(best_code) if bit == 1]
    print("Selected vertices:", selected_vertices)
    print("Fitness:", best_fitness)

Generation 1: Gen: 000110 | Fitness: 10 | Selected: 2
Generation 2: Gen: 000110 | Fitness: 10 | Selected: 2
Generation 3: Gen: 000110 | Fitness: 10 | Selected: 2
Generation 4: Gen: 000110 | Fitness: 10 | Selected: 2
Generation 5: Gen: 000110 | Fitness: 10 | Selected: 2
Generation 6: Gen: 000110 | Fitness: 10 | Selected: 2
Generation 7: Gen: 000110 | Fitness: 10 | Selected: 2
Generation 8: Gen: 000110 | Fitness: 10 | Selected: 2
Generation 9: Gen: 000110 | Fitness: 10 | Selected: 2
Generation 10: Gen: 000110 | Fitness: 10 | Selected: 2
Generation 11: Gen: 000110 | Fitness: 10 | Selected: 2
Generation 12: Gen: 000110 | Fitness: 10 | Selected: 2
Generation 13: Gen: 000110 | Fitness: 10 | Selected: 2
Generation 14: Gen: 000110 | Fitness: 10 | Selected: 2
Generation 15: Gen: 000110 | Fitness: 10 | Selected: 2
Generation 16: Gen: 000110 | Fitness: 10 | Selected: 2
Generation 17: Gen: 000110 | Fitness: 10 | Selected: 2
Generation 18: Gen: 000110 | Fitness: 10 | Selected: 2
Generation 19: Gen:

In [56]:
# NOTE: Uspeo sam da dobijem i [1, 3], ali oba su resenja ispravna i minimalna